In [78]:
import imgaug

In [79]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
import mrcnn.utils as utils
from mrcnn import visualize
from mrcnn.model import log

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "PROJECT/food/mask_rcnn_coco.h5")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [80]:
ROOT_DIR

'/media/thang0912/CE72AE4B72AE385B/rubikAI'

In [81]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [82]:
# # Khởi tạo đối tượng dataset dùng để training mô hình
# # class này kế thừa các thuộc tính của utils.Dataset
# # trong bài này, ta load dataset có sẵn để train mô hình
# # 1.import thư viện có sẵn để xử lý bộ dữ liệu.
# class FoodDataset(utils.Dataset):
#     def load_food(self, dataset_dir, subset):
#         self.add_class("multi class!!!")
#         assert subset in ["train", "val"]
#         dataset_dir = os.path.join(dataset_dir, subset)
#         ## load annotation file
#         with open(os.path.join(dataset_dir, 'annotations.json'), 'r') as f:
#             annotations = json.load(f)
#         annotations = list(annotations.values())
        

In [83]:
class CocoDataset(utils.Dataset):
    def load_coco(self, dataset_dir, year=None, class_ids=None,
                  class_map=None, return_coco=False):
        """Load a subset of the COCO dataset.
        dataset_dir: The root directory of the COCO dataset.
        subset: What to load (train, val, minival, valminusminival)
        year: What dataset year to load (2014, 2017) as a string, not an integer
        class_ids: If provided, only loads images that have the given classes.
        class_map: TODO: Not implemented yet. Supports maping classes from
            different datasets to the same class ID.
        return_coco: If True, returns the COCO object.
        auto_download: Automatically download and unzip MS-COCO images and annotations
        """

        coco = COCO("{}/annotations.json".format(dataset_dir))

        # Load all classes or a subset?
        if not class_ids:
            # All classes
            class_ids = sorted(coco.getCatIds())

        # All images or a subset?
        if class_ids:
            image_ids = []
            for id in class_ids:
                image_ids.extend(list(coco.getImgIds(catIds=[id]))) # error is here
            # Remove duplicates
            image_ids = list(set(image_ids))
        else:
            # All images
            image_ids = list(coco.imgs.keys())

        # Add classes
        for i in class_ids:
            self.add_class("coco", i, coco.loadCats(i)[0]["name"])

        # Add images
        for i in image_ids:
            self.add_image(
                "coco", image_id=i,
                path=os.path.join(image_dir, coco.imgs[i]['file_name']),
                width=coco.imgs[i]["width"],
                height=coco.imgs[i]["height"],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))
        if return_coco:
            return coco

##################################
    def load_mask(self, image_id):
        """Load instance masks for the given image.

        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "coco":
            return super(CocoDataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "coco.{}".format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(CocoDataset, self).load_mask(image_id)

    def image_reference(self, image_id):
        """Return a link to the image in the COCO Website."""
        info = self.image_info[image_id]
        if info["source"] == "coco":
            return "http://cocodataset.org/#explore?id={}".format(info["id"])
        else:
            super(CocoDataset, self).image_reference(image_id)

    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m


In [84]:
def build_coco_results(dataset, image_ids, rois, class_ids, scores, masks):
    """Arrange resutls to match COCO specs in http://cocodataset.org/#format
    """
    # If no results, return an empty list
    if rois is None:
        return []

    results = []
    for image_id in image_ids:
        # Loop through detections
        for i in range(rois.shape[0]):
            class_id = class_ids[i]
            score = scores[i]
            bbox = np.around(rois[i], 1)
            mask = masks[:, :, i]

            result = {
                "image_id": image_id,
                "category_id": dataset.get_source_class_id(class_id, "coco"),
                "bbox": [bbox[1], bbox[0], bbox[3] - bbox[1], bbox[2] - bbox[0]],
                "score": score,
                "segmentation": maskUtils.encode(np.asfortranarray(mask))
            }
            results.append(result)
    return results

In [85]:
def evaluate_coco(model, dataset, coco, eval_type="bbox", limit=0, image_ids=None):
    """Runs official COCO evaluation.
    dataset: A Dataset object with valiadtion data
    eval_type: "bbox" or "segm" for bounding box or segmentation evaluation
    limit: if not 0, it's the number of images to use for evaluation
    """
    # Pick COCO images from the dataset
    image_ids = image_ids or dataset.image_ids

    # Limit to a subset
    if limit:
        image_ids = image_ids[:limit]

    # Get corresponding COCO image IDs.
    coco_image_ids = [dataset.image_info[id]["id"] for id in image_ids]

    t_prediction = 0
    t_start = time.time()

    results = []
    for i, image_id in enumerate(image_ids):
        # Load image
        image = dataset.load_image(image_id)

        # Run detection
        t = time.time()
        r = model.detect([image], verbose=0)[0]
        t_prediction += (time.time() - t)

        # Convert results to COCO format
        # Cast masks to uint8 because COCO tools errors out on bool
        image_results = build_coco_results(dataset, coco_image_ids[i:i + 1],
                                           r["rois"], r["class_ids"],
                                           r["scores"],
                                           r["masks"].astype(np.uint8))
        results.extend(image_results)

    # Load results. This modifies results with additional attributes.
    coco_results = coco.loadRes(results)

    # Evaluate
    cocoEval = COCOeval(coco, coco_results, eval_type)
    cocoEval.params.imgIds = coco_image_ids
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()

    print("Prediction time: {}. Average {}/image".format(
        t_prediction, t_prediction / len(image_ids)))
    print("Total time: ", time.time() - t_start)

In [86]:
# training
class FoodConfig(Config):
    NAME = "food"
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 40
    GPU_COUNT = 1
    DETECTION_MIN_CONFIDENCE = 0

In [87]:
config = FoodConfig()

In [88]:
# created model
model = modellib.MaskRCNN(mode='training',
                          config=config,
                         model_dir=MODEL_DIR)

In [89]:
# load weight pre-trained cho mode
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=['mrcnn_bbox_fc', 'mrcnn_class_logits', 'mrcnn_mask'])

In [90]:
# prepare dataset for training
dataset_training = CocoDataset()

In [91]:
dataset_dir = os.path.join(ROOT_DIR, 'PROJECT/food/train')
image_dir = os.path.join(ROOT_DIR, 'PROJECT/food/train/images')

In [92]:
dataset_training.load_coco(dataset_dir=dataset_dir)

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


In [93]:
dataset_training.prepare()

In [94]:
# khởi tạo đối tượng dataset_train
dataset_train_dir = os.path.join(ROOT_DIR, 'PROJECT/food/train')
image_train_dir = os.path.join(ROOT_DIR, 'PROJECT/food/train/images')
dataset_train = CocoDataset()
dataset_train.load_coco(dataset_dir=dataset_train_dir, return_coco = True)
dataset_train.prepare

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


<bound method Dataset.prepare of <__main__.CocoDataset object at 0x7f0c85c1c438>>

In [62]:
# Khởi tạo đối tượng dataset_val
dataset_val_dir = os.path.join(ROOT_DIR, 'PROJECT/food/val')
image_val_dir = os.path.join(ROOT_DIR, 'PROJECT/food/val/images')
dataset_val = CocoDataset()
dataset_val.load_coco(dataset_dir=dataset_val_dir)
dataset_val.prepare()

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [63]:
np.arange(len(dataset_train.image_info))

array([   0,    1,    2, ..., 5542, 5543, 5544])

In [95]:
import numpy
numpy.random.bit_generator = numpy.random._bit_generator

In [96]:
augmentation = imgaug.augmenters.Fliplr(0.5)

In [66]:
# from tensorflow.python.eager import function

In [97]:
print("Training network heads")

model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads',
            augmentation=augmentation)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /media/thang0912/CE72AE4B72AE385B/rubikAI/logs/food20200125T1534/mask_rcnn_food_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_c

/home/thang0912/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/thang0912/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/thang0912/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/thang0912/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator wit

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/035944.jpg'ERROR:root:Error processing image {'id': 35944, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/035944.jpg', 'width': 607, 'height': 607, 'annotations': [{'id': 64031, 'image_id': 35944, 'category_id': 2578, 'segmentation': [[316.0, 480.5, 270.0, 480.5, 269.0, 479.5, 246.0, 479.5, 245.0, 478.5, 222.0, 478.5, 221.0, 477.5, 217.0, 477.5, 216.0, 476.5, 214.0, 476.5, 213.0, 475.5, 212.00000000000003, 475.5, 211.0, 474.5, 210.00000000000003, 474.5, 209.0, 473.5, 208.00000000000003, 473.5, 207.0, 472.5, 206.00000000000003, 472.5, 205.0, 471.5, 204.00000000000003, 471.5, 199.5, 466.99999999999994, 199.5, 466.0, 196.5, 462.99999999999994, 196.5, 462.0, 194.5, 460.0, 194.5, 458.0, 193.5, 457.0, 193.5, 456.0, 192.5, 454.99999999999994, 192.5, 453.0, 191.5, 452.0, 191.5, 450.0, 190.5, 449.0, 190.5, 447.0, 189.5, 446.0, 189.5, 444.0, 188.5,

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/035944.jpg'ERROR:root:Error processing image {'id': 35944, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/035944.jpg', 'width': 607, 'height': 607, 'annotations': [{'id': 64031, 'image_id': 35944, 'category_id': 2578, 'segmentation': [[316.0, 480.5, 270.0, 480.5, 269.0, 479.5, 246.0, 479.5, 245.0, 478.5, 222.0, 478.5, 221.0, 477.5, 217.0, 477.5, 216.0, 476.5, 214.0, 476.5, 213.0, 475.5, 212.00000000000003, 475.5, 211.0, 474.5, 210.00000000000003, 474.5, 209.0, 473.5, 208.00000000000003, 473.5, 207.0, 472.5, 206.00000000000003, 472.5, 205.0, 471.5, 204.00000000000003, 471.5, 199.5, 466.99999999999994, 199.5, 466.0, 196.5, 462.99999999999994, 196.5, 462.0, 194.5, 460.0, 194.5, 458.0, 193.5, 457.0, 193.5, 456.0, 192.5, 454.99999999999994, 192.5, 453.0, 191.5, 452.0, 191.5, 450.0, 190.5, 449.0, 190.5, 447.0, 189.5, 446.0, 189.5, 444.0, 188.5,

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/009700.jpg'
ERROR:root:Error processing image {'id': 9700, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/009700.jpg', 'width': 479, 'height': 479, 'annotations': [{'id': 6407, 'image_id': 9700, 'category_id': 2053, 'segmentation': [[155.675, 164.0575, 204.7725, 137.71249999999998, 140.1075, 104.1825, 93.405, 156.8725, 98.195, 197.58749999999998, 110.17, 265.845, 105.38, 325.72, 150.885, 350.8675, 183.2175, 342.48499999999996, 219.1425, 292.19, 256.26500000000004, 293.38750000000005, 334.1025, 341.2875, 385.595, 338.8925, 359.25, 261.055, 237.105, 182.02, 213.155, 166.4525, 164.0575, 161.66250000000002]], 'area': 38229.0, 'bbox': [93.405, 93.405, 257.4625, 292.19000000000005], 'iscrowd': 0}, {'id': 6406, 'image_id': 9700, 'category_id': 1566, 'segmentation': [[122.145, 73.0475, 70.65249999999999, 89.8125, 67.06, 136.515, 33.53, 164.0575, 

...

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/036445.jpg'



ERROR:root:Error processing image {'id': 9791, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/009791.jpg', 'width': 943, 'height': 943, 'annotations': [{'id': 6717, 'image_id': 9791, 'category_id': 2578, 'segmentation': [[202.745, 339.47999999999996, 259.32500000000005, 584.66, 363.055, 704.8925, 464.4275, 742.6125, 546.9399999999999, 711.965, 676.6025000000001, 627.095, 744.97, 405.49, 719.0374999999999, 304.1175, 603.52, 223.96249999999998, 372.485, 219.2475, 261.6825, 245.18, 205.1025, 330.04999999999995]], 'area': 214776.0, 'bbox': [202.745, 202.745, 539.8675, 542.225], 'iscrowd': 0}]}
Traceback (most recent call last):
  File "/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJ

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/015970.jpg'ERROR:root:Error processing image {'id': 15970, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/015970.jpg', 'width': 479, 'height': 479, 'annotations': [{'id': 20339, 'image_id': 15970, 'category_id': 1070, 'segmentation': [[241.99999999999997, 368.5, 217.0, 368.5, 216.0, 367.5, 201.0, 367.5, 200.0, 366.5, 187.0, 366.5, 186.0, 365.5, 172.0, 365.5, 171.0, 364.5, 163.0, 364.5, 162.0, 363.5, 158.0, 363.5, 157.0, 362.5, 155.0, 362.5, 154.0, 361.5, 153.0, 361.5, 152.0, 360.5, 151.0, 360.5, 150.0, 359.5, 149.0, 359.5, 148.0, 358.5, 146.0, 358.5, 145.0, 357.5, 142.0, 357.5, 141.0, 356.5, 138.0, 356.5, 137.0, 355.5, 134.0, 355.5, 133.0, 354.5, 131.0, 354.5, 130.0, 353.5, 127.99999999999999, 353.5, 127.0, 352.5, 126.0, 352.5, 123.99999999999999, 350.5, 123.0, 350.5, 120.99999999999999, 348.5, 120.0, 348.5, 119.0, 347.5, 118.0, 347.5, 11

FileNotFoundError: No such file: '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/009791.jpg'


ERROR:root:Error processing image {'id': 9791, 'source': 'coco', 'path': '/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/train/images/009791.jpg', 'width': 943, 'height': 943, 'annotations': [{'id': 6717, 'image_id': 9791, 'category_id': 2578, 'segmentation': [[202.745, 339.47999999999996, 259.32500000000005, 584.66, 363.055, 704.8925, 464.4275, 742.6125, 546.9399999999999, 711.965, 676.6025000000001, 627.095, 744.97, 405.49, 719.0374999999999, 304.1175, 603.52, 223.96249999999998, 372.485, 219.2475, 261.6825, 245.18, 205.1025, 330.04999999999995]], 'area': 214776.0, 'bbox': [202.745, 202.745, 539.8675, 542.225], 'iscrowd': 0}]}
Traceback (most recent call last):
  File "/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJECT/food/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/media/thang0912/CE72AE4B72AE385B/rubikAI/PROJE

Epoch 1/1


UnboundLocalError: local variable 'image_id' referenced before assignment

In [ ]:
# training - 3 stages
# Training - Stage 1
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='heads',
                augmentation=augmentation)

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=120,
                layers='4+',
                augmentation=augmentation)

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=160,
                layers='all',
                augmentation=augmentation)

In [ ]:
# evaluate model

# prepare val dataset
# load val dataset
# evaluate with model


In [39]:
import json

In [40]:
with open(os.path.join(ROOT_DIR, 'PROJECT/food/train/annotations.json')) as f:
    data = json.load(f)